In [ ]:
OPENROUTER_API_KEY = ""

MODEL_NAME = "mistralai/mistral-nemo" #Modelo de IA do OpenRouter
APP_NAME = 'IA de Receitas'

# Limites gerais
MAX_OUTPUT_TOKENS = 2000 # tokens de saída (resposta)
MAX_CONTEXT_CHARS = 10000 # limite simples por caracteres para o contexto enviado

# Importações
import requests
import json
import matplotlib.pyplot as plt

import time

!pip install ddgs -q # Instala a biblioteca Duck Duck Go e silencia os avisos de instalação
from ddgs import DDGS

github = 'https://github.com/luanbalves-40/projeto-ia'
historico = ''

def pesquisar(pesquisa: str):
  print("Pesquisando...")
  texto_pesquisa = ""

  with DDGS() as ddgs:
    results = ddgs.text(pesquisa, max_results=5)

  if results:
    texto_pesquisa+=f"""
      Pesquisa: {pesquisa}
      Resultados:\n
    """

  for r in results:
    texto_pesquisa+=f"{r["title"]}\n{r["body"]}\n\n"

  return texto_pesquisa

def pesquisa_necessaria(pergunta: str):
  ie = """
  Caso você consiga responder à pergunta sem pesquisar, RESPONDA EXATAMENTE 'False' (sem as aspas).
  Caso contrário, responda APENAS com a pesquisa que quer fazer (exemplo: como fazer bolo de chocolate?).
  """

  p = perguntar_openrouter(pergunta,'',ie)
  if p == "False":
    return False
  else:
    return p


def perguntar_openrouter(pergunta: str,
                         pesquisa: str,
                         instrucoes_especiais: str
                         ) -> str:
    """Envia a pergunta com historico ao endpoint do OpenRouter."""
    if not OPENROUTER_API_KEY or not OPENROUTER_API_KEY.startswith("sk-"):
        return "⚠️ Configure OPENROUTER_API_KEY corretamente."

    system_msg = (
        "Você é um assistente IA para receitas"
    )

    if instrucoes_especiais == False:
      user_prompt = f"""

      Instruções:
      1: O Histórico é o registro de mensagens entre você (Modelo) e o usuário (Usuário);
      2: A Pergunta é a mensagem que o usuário deseja que o Modelo responda. Responda em português, seja objetivo e simpático. Responda apenas o que for perguntado;
      3: As Pesquisas são possíveis pesquisas sobre a pergunta do usuário, caso este campo esteja vazio Pesquisas são foram necessárias então você poderá ignorar este campo;
      4: Não use Markdown, não use '#', não use '*', não use negrito nem títulos. Responda apenas em texto corrido, sem formatação.

      [Histórico]
      {historico}

      [PERGUNTA]
      {pergunta}

      [PESQUISAS]
      {pesquisa}
      """.strip()
    else:
      user_prompt = f"""
      Instruções:
      {instrucoes_especiais}

      [HISTÓRICO]
      {historico}

      [PERGUNTA]
      {pergunta}

      [PESQUISAS]
      {pesquisa}
      """
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": github,
        "X-Title": APP_NAME,
    }

    body = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_prompt},
        ],
        "max_tokens": MAX_OUTPUT_TOKENS,
    }

    try:
        resp = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=body,          # usa json=body (requests serializa pra você)
            timeout=90,
        )
        resp.raise_for_status()
        data = resp.json()
        return data["choices"][0]["message"]["content"].strip()
    except requests.HTTPError as e:
        try:
            err = resp.json()
        except Exception:
            err = {"error": str(e)}
        return f"❌ Erro HTTP: {e}\nDetalhes: {err}"
    except Exception as e:
        return f"❌ Erro: {e}"

print(f"Bem-vindo ao seu Assistente de Receitas IA!\n\nInstruções:\nBasta perguntar para receber informações detalhadas dos seus pratos favoritos!\n")

while True:
  msg_usuario = input("Pergunte ou digite '1' para sair: ")
  tempo_inicial = time.time()

  if msg_usuario == "1":
    break
  p = pesquisa_necessaria(msg_usuario)

  if p == False:
    resposta = perguntar_openrouter(msg_usuario,'',False)
  else:
    resposta = perguntar_openrouter(msg_usuario,pesquisar(p),False)

  print(f"Tempo de resposta: {time.time() - tempo_inicial:.2f} segundos")
  print(resposta)
  historico+=f"Usuário: {msg_usuario}\n"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.2 MB/s eta 0:00:00
Bem-vindo ao seu Assistente de Receitas IA!

Instruções:
Basta perguntar para receber informações detalhadas dos seus pratos favoritos!

Pergunte ou digite '1' para sair: Como fazer bolo de fubá?
Pesquisando...
Tempo de resposta: 13.27 segundos
Para fazer um bolo de fubá, você vai precisar dos seguintes ingredientes:

* 2 xícaras de fubá de milho
* 1 xícara de açúcar
* 3 ovos
* 1 xícara de leite
* 1/2 xícara de óleo
* 1 colher de chá de fermento em pó
* 1/2 colher de chá de sal

Instruções:

1. Pré-aqueça o forno a 180°C e unte uma forma de bolo com manteiga e farinha.
2. Em uma tigela, misture o fubá, o açúcar, o sal e o fermento.
3. Em outra tigela, bata os ovos, o leite e o óleo até ficar bem misturado.
4. Junte a mistura de ovos ao fubá e misture bem 

KeyboardInterrupt: Interrupted by user